In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as soup
from urllib.request import Request, urlopen
import json
from tqdm import tqdm

In [18]:
req_list = []
req_list.append(Request('https://www.moneycontrol.com/mutual-funds/nav/aditya-birla-sun-life-low-duration-fund-direct-plan/MBS919', headers={'User-Agent': 'Mozilla/5.0'}))
req_list.append(Request('https://www.moneycontrol.com/mutual-funds/nav/axis-bluechip-fund-direct-plan/MAA181', headers={'User-Agent': 'Mozilla/5.0'}))
req_list.append(Request('https://www.moneycontrol.com/mutual-funds/nav/mirae-asset-large-cap-fund-direct-plan/MMA096', headers={'User-Agent': 'Mozilla/5.0'}))
req_list.append(Request('https://www.moneycontrol.com/mutual-funds/nav/sbi-magnum-multicap-fund-direct-plan/MSB503', headers={'User-Agent': 'Mozilla/5.0'}))
req_list.append(Request('https://www.moneycontrol.com/mutual-funds/nav/icici-prudential-bluechip-fund-direct-plan/MPI1134', headers={'User-Agent': 'Mozilla/5.0'}))
req_list.append(Request('https://www.moneycontrol.com/mutual-funds/nav/motilal-oswal-multicap-35-fund-direct-plan/MMO031', headers={'User-Agent': 'Mozilla/5.0'}))
req_list.append(Request('https://www.moneycontrol.com/mutual-funds/nav/nippon-india-large-cap-fund-direct-plan/MRC940', headers={'User-Agent': 'Mozilla/5.0'}))


In [19]:
webpage_list = []
for req in tqdm(req_list):    
    client = urlopen(req)
    webpage_list.append(client.read())
    client.close()

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:04<00:00,  1.73it/s]


In [20]:
page_soup_list = []
for webpage in tqdm(webpage_list):
    page_soup_list.append(soup(webpage,"html.parser"))

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:02<00:00,  3.42it/s]


In [21]:
#Fund Names:
fund_name = []
for page_soup in page_soup_list:
    fund_name.append(page_soup.find("h1",{"class":"page_heading"}).text)
fund_name

['Aditya Birla Sun Life Low Duration Fund - Direct Plan - Growth',
 'Axis Bluechip Fund - Direct Plan - Growth',
 'Mirae Asset Large Cap Fund - Direct Plan - Growth',
 'SBI Magnum MultiCap Fund - Direct Plan - Growth',
 'ICICI Prudential Bluechip Fund - Direct Plan - Growth',
 'Motilal Oswal Multicap 35 Fund - Direct Plan - Growth',
 'Nippon India Large Cap Fund - Direct Plan - Growth']

In [ ]:
#01 - Getting isin and ind_id:

In [22]:
isin = []
ind_id = []
for page_soup in page_soup_list:
    isin.append(page_soup.findAll("li",{"id":"li_1m"})[0].a['onclick'][17:29])
    ind_id.append(page_soup.findAll("script")[46].text[1506:1508])

In [23]:
print(isin)
print(ind_id)


['INF209K01XU7', 'INF846K01DP8', 'INF769K01AX2', 'INF200K01UG1', 'INF109K016L0', 'INF247L01502', 'INF204K01XI3']
['82', '82', '82', '82', '82', '82', '82']


'INF209K01XU7'

In [24]:
#Preparing data link:
link_list = []
for i in range(len(isin)):
    link_list.append('https://www.moneycontrol.com/mc/widget/mfnavonetimeinvestment/get_chart_value?isin='+isin[i]+'&dur=ALL&ind_id='+ind_id[i]+'&indexName=CRISIL%2010%20Year%20Gilt%20Index&classic=true&type=benchmark&investmentType=Debt')
    

In [25]:
#Getting data page list:
webpage2_list = []
for link in link_list:
    req2 = Request(link, headers={'User-Agent': 'Mozilla/5.0'})
    client = urlopen(req2)
    webpage2_list.append(client.read())
    client.close()

In [26]:
#Getting data webpages:
m_list = []
for webpage in webpage2_list:
    z = str(soup(webpage,"html.parser"))
    m_list.append(json.loads(z))

In [27]:
len(m_list)

7

In [28]:
#getting date and nav of all MFs.
date = []
nav = []
for i in range(len(m_list)):
    date2 = []
    nav2 = []
    for x in m_list[i]['g1']:
        date2.append(x['navDate'])
        nav2.append(x['navValue'])
    date.append(date2)
    nav.append(nav2)
    date[i].reverse()
    nav[i].reverse()

In [29]:
#Getting max length:
max = 0
for d in date:
    if(max<len(d)):
        max = len(d)
max

1700

In [30]:
# Making all list of same size:
for i in range(len(date)):
    blanks = max-len(date[i])
    date[i][len(date[i]):max] = blanks * [0]
    nav[i][len(nav[i]):max] = blanks * [0]

In [31]:
df = pd.DataFrame()
for i in range(len(date)):
    date_for = 'Date_'+fund_name[i][0:11].replace(' ','_')
    nav_for = 'NAV_'+fund_name[i][0:11].replace(' ','_')
    df[date_for] = date[i]
    df[nav_for] = nav[i]

In [32]:
df.tail()

,Date_Aditya_Birl,NAV_Aditya_Birl,Date_Axis_Bluech,NAV_Axis_Bluech,Date_Mirae_Asset,NAV_Mirae_Asset,Date_SBI_Magnum_,NAV_SBI_Magnum_,Date_ICICI_Prude,NAV_ICICI_Prude,Date_Motilal_Osw,NAV_Motilal_Osw,Date_Nippon_Indi,NAV_Nippon_Indi
1695,0,0.0,2013-01-04,12.35,2013-01-07,18.950,2013-01-07,18.99,2013-01-04,18.76,0,0.0,2013-01-08,14.7197
1696,0,0.0,2013-01-03,12.32,2013-01-04,19.008,0,0.00,2013-01-03,18.73,0,0.0,2013-01-07,14.7718
1697,0,0.0,2013-01-02,12.28,2013-01-03,19.011,0,0.00,2013-01-02,18.66,0,0.0,2013-01-04,14.8037
1698,0,0.0,0,0.00,2013-01-02,18.972,0,0.00,2013-01-01,18.55,0,0.0,2013-01-03,14.7305
1699,0,0.0,0,0.00,0,0.000,0,0.00,0,0.00,0,0.0,2013-01-02,14.6967


In [33]:
df.to_csv('all_MFs.csv',index=False)